In [191]:
import pandas as pd
import numpy as np
import re

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

In [192]:
finance = pd.read_csv('./datasets/finance.csv')
finance = finance.rename(columns={"headline":"headline_en", "translated_text": "headline_ru"})
finance

,sentiment,headline_en,headline_ru
0,neutral,"According to Gran , the company has no plans t...","По словам Граня, компания не планирует перенос..."
1,neutral,Technopolis plans to develop in stages an area...,«Технополис» планирует поэтапно застроить терр...
2,negative,The international electronic industry company ...,Международная компания электронной промышленно...
3,positive,With the new production plant the company woul...,Благодаря новому производственному предприятию...
4,positive,According to the company 's updated strategy f...,Согласно обновленной стратегии компании на 200...
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,LONDON MarketWatch – Цены на акции в Лондоне в...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,"Продажи пива Rinkuskiai упали на 6,5% до 4,16 ..."
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,"Операционная прибыль упала до 35,4 млн евро с ..."
4844,negative,Net sales of the Paper segment decreased to EU...,Чистые продажи бумажного сегмента снизились до...


In [193]:
gpt_train = pd.read_csv('./datasets/gpt_train.csv')
gpt_train = gpt_train.rename(columns={"headline":"headline_en", "translated_text": "headline_ru"})
gpt_train

,headline_en,sentiment,headline_ru
0,Investors rejoice at the record growth of bitcoin,positive,Инвесторы радуются рекордному росту биткоина
1,Experts warn about the risks of investing in c...,negative,Эксперты предупреждают о рисках инвестирования...
2,"Bitcoin is updating its maximum, attracting ne...",positive,"Биткойн обновляет свой максимум, привлекая нов..."
3,Bitcoin transactions are subject to cyber attacks,negative,Биткойн-транзакции подвергаются кибератакам
4,Developers have introduced new technologies to...,positive,Разработчики представили новые технологии для ...
...,...,...,...
999,Investors assess the role of cryptocurrencies ...,positive,Инвесторы оценивают роль криптовалют в содейст...
1000,Cryptocurrencies are gaining popularity in the...,positive,Криптовалюты набирают популярность в сфере отв...
1001,Bitcoin is becoming part of a long-term invest...,positive,Биткойн становится частью долгосрочной инвести...
1002,Experts are considering the impact of cryptocu...,neutral,Эксперты рассматривают влияние криптовалют на ...


In [194]:
risks = pd.read_csv('./datasets/risks.csv')
risks = risks.rename(columns={"headline":"headline_en", "translated_text": "headline_ru"})
risks

,headline_en,sentiment,headline_ru
0,The GeoSolutions technology will leverage Bene...,positive,Технология GeoSolutions будет использовать GPS...
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative,"$ESI находится на минимуме, падение с $1,50 до..."
2,"For the last quarter of 2010 , Componenta 's n...",positive,За последний квартал 2010 года чистый объем пр...
3,According to the Finnish-Russian Chamber of Co...,neutral,"По данным Финско-Российской торговой палаты, в..."
4,The Swedish buyout firm has sold its remaining...,neutral,Шведская фирма по выкупу продала оставшиеся 22...
...,...,...,...
5837,RISING costs have forced packaging producer Hu...,negative,РОСТ затрат вынудил производителя упаковки Huh...
5838,Nordic Walking was first used as a summer trai...,neutral,Скандинавская ходьба впервые использовалась в ...
5839,"According shipping company Viking Line , the E...",neutral,"По мнению судоходной компании Viking Line, реш..."
5840,"In the building and home improvement trade , s...",neutral,В сфере строительства и товаров для дома прода...


In [195]:
user_responses = pd.read_csv('./datasets/user_respsonses.csv')
user_responses = user_responses.rename(columns={"headline":"headline_en", "translated_text": "headline_ru"})
user_responses

,headline_en,sentiment,headline_ru
0,It isn't an investment - it's speculation in a...,negative,Это не инвестиции – это спекуляция в финансово...
1,Joke of the decade,negative,Шутка десятилетия
2,This is pretty big news. Love to see utility e...,positive,Это довольно большая новость. Приятно видеть п...
3,"Just hold your bag and have an exit strategy, ...",positive,Просто держите свою сумку и имейте стратегию в...
4,While you can trace a common thread from the h...,negative,Хотя вы можете проследить общую нить от контрк...
...,...,...,...
557,Glorified ponzi scheme that wastes people's ti...,negative,"Прославленная финансовая пирамида, которая тра..."
558,Axie will fail because it’s just a mediocre ga...,negative,"Axie потерпит неудачу, потому что это всего ли..."
559,I do not feel sorry for anyone who got into Po...,negative,"Мне не жаль никого, кто попал в ПонциКвон II"
560,good time to accumulate,positive,хорошее время для накопления


In [196]:
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))

def remove_urls(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def lower(text):
    low_text= text.lower()
    return low_text

def remove_num(text):
    remove= re.sub(r'\d+', '', text)
    return remove

def punct_remove(text):
    punct = re.sub(r"[^\w\s\d]","", text)
    return punct

def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def remove_mention(x):
    text=re.sub(r'@\w+','',x)
    return text

def remove_hash(x):
    text=re.sub(r'#\w+','',x)
    return text

def remove_space(text):
    space_remove = re.sub(r"\s+"," ",text).strip()
    return space_remove

In [197]:
", ".join(stopwords.words('russian'))
STOPWORDS = set(stopwords.words('russian'))

def remove_urls_ru(text):
    url_remove = re.compile(r'https?://\S+|www\.\S+')
    return url_remove.sub(r'', text)

def remove_html_ru(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

def lower_ru(text):
    low_text= text.lower()
    return low_text

def remove_num_ru(text):
    remove= re.sub(r'\d+', '', text)
    return remove

def punct_remove_ru(text):
    punct = re.sub(r"[^\w\s\d]","", text)
    return punct

def remove_stopwords_ru(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

def remove_mention_ru(x):
    text=re.sub(r'@\w+','',x)
    return text

def remove_hash_ru(x):
    text=re.sub(r'#\w+','',x)
    return text

def remove_space_ru(text):
    space_remove = re.sub(r"\s+"," ",text).strip()
    return space_remove

In [198]:
df = pd.concat([finance, gpt_train, user_responses, risks], ignore_index=True)
df

,sentiment,headline_en,headline_ru
0,neutral,"According to Gran , the company has no plans t...","По словам Граня, компания не планирует перенос..."
1,neutral,Technopolis plans to develop in stages an area...,«Технополис» планирует поэтапно застроить терр...
2,negative,The international electronic industry company ...,Международная компания электронной промышленно...
3,positive,With the new production plant the company woul...,Благодаря новому производственному предприятию...
4,positive,According to the company 's updated strategy f...,Согласно обновленной стратегии компании на 200...
...,...,...,...
12249,negative,RISING costs have forced packaging producer Hu...,РОСТ затрат вынудил производителя упаковки Huh...
12250,neutral,Nordic Walking was first used as a summer trai...,Скандинавская ходьба впервые использовалась в ...
12251,neutral,"According shipping company Viking Line , the E...","По мнению судоходной компании Viking Line, реш..."
12252,neutral,"In the building and home improvement trade , s...",В сфере строительства и товаров для дома прода...


In [199]:
def nlp_process_ru(df):
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_urls_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_html_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:lower_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_num_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:punct_remove_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_stopwords_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_mention_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_hash_ru(x))
    df['headline_ru']=df['headline_ru'].apply(lambda x:remove_space_ru(x))
    return df

def nlp_process(df):
    df['headline_en']=df['headline_en'].apply(lambda x:remove_urls(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_html(x))
    df['headline_en']=df['headline_en'].apply(lambda x:lower(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_num(x))
    df['headline_en']=df['headline_en'].apply(lambda x:punct_remove(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_stopwords(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_mention(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_hash(x))
    df['headline_en']=df['headline_en'].apply(lambda x:remove_space(x))
    return df

In [200]:
df = nlp_process(df)
df = nlp_process_ru(df)
df

,sentiment,headline_en,headline_ru
0,neutral,according to gran the company has no plans to ...,словам граня компания планирует переносить про...
1,neutral,technopolis plans to develop in stages an area...,технополис планирует поэтапно застроить террит...
2,negative,the international electronic industry company ...,международная компания электронной промышленно...
3,positive,with the new production plant the company woul...,благодаря новому производственному предприятию...
4,positive,according to the company s updated strategy fo...,согласно обновленной стратегии компании годы b...
...,...,...,...
12249,negative,rising costs have forced packaging producer hu...,рост затрат вынудил производителя упаковки huh...
12250,neutral,nordic walking was first used as a summer trai...,скандинавская ходьба впервые использовалась ка...
12251,neutral,according shipping company viking line the eu ...,мнению судоходной компании viking line решение...
12252,neutral,in the building and home improvement trade sal...,сфере строительства товаров дома продажи снизи...


In [201]:
ru_df = df[['headline_ru', 'sentiment']]
en_df = df[['headline_en', 'sentiment']]
df = pd.DataFrame({"headline": pd.concat([en_df['headline_en'], ru_df['headline_ru']], ignore_index=True), "sentiment": pd.concat([en_df['sentiment'], ru_df['sentiment']], ignore_index=True)})
df

,headline,sentiment
0,according to gran the company has no plans to ...,neutral
1,technopolis plans to develop in stages an area...,neutral
2,the international electronic industry company ...,negative
3,with the new production plant the company woul...,positive
4,according to the company s updated strategy fo...,positive
...,...,...
24503,рост затрат вынудил производителя упаковки huh...,negative
24504,скандинавская ходьба впервые использовалась ка...,neutral
24505,мнению судоходной компании viking line решение...,neutral
24506,сфере строительства товаров дома продажи снизи...,neutral


In [202]:
df = df.dropna()
null_count = df.isnull().sum()
null_count

headline     0
sentiment    0
dtype: int64

In [203]:
df = df.iloc[np.random.permutation(df.index)].reset_index(drop=True)
df

,headline,sentiment
0,the purchase of the operations is part of yit ...,positive
1,because its based on nothing but belief that i...,negative
2,with this the company will exit the contract m...,neutral
3,based in helsinki finland ramirent has branche...,neutral
4,это действительно поднимает эффективность новы...,positive
...,...,...
24503,транспортный бизнес компании осуществляется ко...,neutral
24504,эта информация опубликована сентября года амер...,neutral
24505,комиссионный доход снизился млн евро млн евро ...,neutral
24506,september finnish waste management and recycli...,positive


In [204]:
X = df['headline']
y = df['sentiment']

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32, input_length=X_padded.shape[1]))
model.add(GlobalAveragePooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax')) 

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=64, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")
model.save('sentiment_model_v5.keras')

Epoch 1/50
246/246 [==============================] - 2s 6ms/step - loss: 1.0021 - accuracy: 0.5184 - val_loss: 0.9821 - val_accuracy: 0.5217
Epoch 2/50
246/246 [==============================] - 2s 6ms/step - loss: 0.9895 - accuracy: 0.5206 - val_loss: 0.9799 - val_accuracy: 0.5217
Epoch 3/50
246/246 [==============================] - 1s 6ms/step - loss: 0.9809 - accuracy: 0.5208 - val_loss: 0.9605 - val_accuracy: 0.5240
Epoch 4/50
246/246 [==============================] - 1s 6ms/step - loss: 0.9228 - accuracy: 0.5545 - val_loss: 0.8775 - val_accuracy: 0.5637
Epoch 5/50
246/246 [==============================] - 1s 6ms/step - loss: 0.8261 - accuracy: 0.6235 - val_loss: 0.8082 - val_accuracy: 0.6448
Epoch 6/50
246/246 [==============================] - 1s 6ms/step - loss: 0.7484 - accuracy: 0.6785 - val_loss: 0.7639 - val_accuracy: 0.6828
Epoch 7/50
246/246 [==============================] - 1s 6ms/step - loss: 0.6828 - accuracy: 0.7123 - val_loss: 0.7277 - val_accuracy: 0.6948
Epoch 

In [207]:
check_df = pd.read_csv('./datasets/check.csv')
check_df = check_df.drop(columns=['id'])
check_df = nlp_process(check_df)
check_df = nlp_process_ru(check_df)
ru_check_df = check_df['headline_ru']
en_check_df = check_df['headline_en']
check_df = pd.DataFrame({"headline": pd.concat([en_check_df, ru_check_df], ignore_index=True)})
check_df

,headline
0,биткоин упал последние часа вызвав тревогу сре...
1,биткоин достиг нового исторического максимума ...
2,биткоин останется таким году
3,президент франции заявляет концу года купит би...
4,последние новостные сообщения отмечают увеличе...
5,последние колебания цене биткоина вызвали опас...
6,технические обновления биткоине продолжаются ц...
7,резкое снижение стоимости биткоина вызвало сер...
8,биткоин получил критику изза своего воздействи...
9,рыночная стоимость биткоина оставалась относит...


In [208]:
X_new = check_df['headline']

X_new_sequences = tokenizer.texts_to_sequences(X_new)
X_new_padded = pad_sequences(X_new_sequences, maxlen=X_padded.shape[1])

saved_model_path = 'sentiment_model_v5.keras'
model = load_model(saved_model_path)

predictions = model.predict(X_new_padded)

predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))

check_df['predicted_sentiment'] = predicted_labels

check_df


1/1 [==============================] - 0s 25ms/step


,headline,predicted_sentiment
0,биткоин упал последние часа вызвав тревогу сре...,negative
1,биткоин достиг нового исторического максимума ...,positive
2,биткоин останется таким году,negative
3,президент франции заявляет концу года купит би...,positive
4,последние новостные сообщения отмечают увеличе...,positive
5,последние колебания цене биткоина вызвали опас...,negative
6,технические обновления биткоине продолжаются ц...,positive
7,резкое снижение стоимости биткоина вызвало сер...,negative
8,биткоин получил критику изза своего воздействи...,negative
9,рыночная стоимость биткоина оставалась относит...,neutral
